# 5.3 Lab: Cross-Validation and the Bootstrap

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

## 5.3.1 The Validation Set Approach

In [2]:
data = pd.read_csv(r'E:\programming\dataset\Into_to_statstical_learning\Auto.csv')
print(data.shape)
data.head()

(397, 9)


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [3]:
#Since we can see the data type of horsepower is object, when it should be numrical
data.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight            int64
acceleration    float64
year              int64
origin            int64
name             object
dtype: object

In [4]:
#by exploring the values of horsepower, it has a "?", due to which the while column is being considered of object datatype
data['horsepower'].unique()

array(['130', '165', '150', '140', '198', '220', '215', '225', '190',
       '170', '160', '95', '97', '85', '88', '46', '87', '90', '113',
       '200', '210', '193', '?', '100', '105', '175', '153', '180', '110',
       '72', '86', '70', '76', '65', '69', '60', '80', '54', '208', '155',
       '112', '92', '145', '137', '158', '167', '94', '107', '230', '49',
       '75', '91', '122', '67', '83', '78', '52', '61', '93', '148',
       '129', '96', '71', '98', '115', '53', '81', '79', '120', '152',
       '102', '108', '68', '58', '149', '89', '63', '48', '66', '139',
       '103', '125', '133', '138', '135', '142', '77', '62', '132', '84',
       '64', '74', '116', '82'], dtype=object)

In [5]:
# removing the points which has '?' value of horsepower, we will first replace it by np.nan
data['horsepower'] = data['horsepower'].replace('?',np.nan)

In [6]:
# than drop all the points which have np.nan
data = data.dropna()

In [7]:
# since now, all the points are numeric, change the datatype to int
data['horsepower'] = data['horsepower'].astype('int')

In [8]:
# for the first part we need to divide the set into two parts
# test_size = 0.5, ie, both the sets have similar number of observations, 196
X_train,X_test,y_train,y_test = train_test_split(data['horsepower'],data['mpg'],test_size = 0.5,random_state = 0)
print(X_train.shape,X_test.shape)

(196,) (196,)


In [9]:
lr = LinearRegression()
lr.fit(X_train.to_frame(),y_train)
# since X_test, is a single column, it will be considered as a series, we need to change it to dataframe
pred = lr.predict(X_test.to_frame())
print('Mean squared error is ',mean_squared_error(y_test,pred))

Mean squared error is  23.616617069669882


In [10]:
# quadratic polynomial
lr = LinearRegression()
# include bias = False, will not return the constant value, that is X**0, as that is added automatically by lin regression
poly = PolynomialFeatures(2,include_bias=False)
poly.fit(X_train.to_frame())
X_train_quad = poly.transform(X_train.to_frame())
X_test_quad = poly.transform(X_test.to_frame())

lr.fit(X_train_quad,y_train)
pred = lr.predict(X_test_quad)

print('Mean squared error for quadratic is ',mean_squared_error(y_test,pred))

Mean squared error for quadratic is  18.76303134689764


In [11]:
#cubic polynomial
lr = LinearRegression()
poly = PolynomialFeatures(3,include_bias=False)
poly.fit(X_train.to_frame())
X_train_cubic = poly.transform(X_train.to_frame())
X_test_cubic = poly.transform(X_test.to_frame())

lr.fit(X_train_cubic,y_train)
pred = lr.predict(X_test_cubic)

print('Mean squared error for cubic is ',mean_squared_error(y_test,pred))

Mean squared error for cubic is  18.79694163263048


In [12]:
# with different random state
# earlier it was random state 0, now its 1
X_train,X_test,y_train,y_test = train_test_split(data['horsepower'],data['mpg'],test_size = 0.5,random_state = 1)
print(X_train.shape,X_test.shape)

lr = LinearRegression()
lr.fit(X_train.to_frame(),y_train)
pred = lr.predict(X_test.to_frame())
print('Mean squared error is ',mean_squared_error(y_test,pred))

# quadratic polynomial
lr = LinearRegression()
poly = PolynomialFeatures(2,include_bias=False)
poly.fit(X_train.to_frame())
X_train_quad = poly.transform(X_train.to_frame())
X_test_quad = poly.transform(X_test.to_frame())

lr.fit(X_train_quad,y_train)
pred = lr.predict(X_test_quad)

print('Mean squared error for quadratic is ',mean_squared_error(y_test,pred))

#cubic polynomial
lr = LinearRegression()
poly = PolynomialFeatures(3,include_bias = False)
poly.fit(X_train.to_frame())
X_train_cubic = poly.transform(X_train.to_frame())
X_test_cubic = poly.transform(X_test.to_frame())

lr.fit(X_train_cubic,y_train)
pred = lr.predict(X_test_cubic)

print('Mean squared error for cubic is ',mean_squared_error(y_test,pred))

(196,) (196,)
Mean squared error is  24.80212062059356
Mean squared error for quadratic is  18.848292603275663
Mean squared error for cubic is  18.805111358604574


## 5.3.2 Leave-One-Out Cross-Validation

In [16]:
error_list = []
for power in range(1,6):
    X = data['horsepower']
    y = data['mpg']
    poly = PolynomialFeatures(power,include_bias=False)
    X = poly.fit_transform(X.to_frame())
    
    lr = LinearRegression()
    # for LOOCV, the number of folds be will n = size of data
    error_list.append(-1*cross_val_score(lr,X,y,cv = len(X),scoring = 'neg_mean_squared_error').mean())

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

In [21]:
pd.DataFrame({"DEGREE":np.arange(1,6),"MEAN SQUARED ERROR":error_list})

,DEGREE,MEAN SQUARED ERROR
0,1,24.231514
1,2,19.248213
2,3,19.334984
3,4,19.424430
4,5,19.033213


## 5.3.3 k-Fold Cross-Validation

In [24]:
error_list = []
for power in range(1,11):
    X = data['horsepower']
    y = data['mpg']
    poly = PolynomialFeatures(power,include_bias=False)
    X = poly.fit_transform(X.to_frame())
    
    lr = LinearRegression()
    error_list.append(-1*cross_val_score(lr,X,y,cv = 10,scoring = 'neg_mean_squared_error').mean())

print('K FOLD CV')    
pd.DataFrame({"DEGREE":np.arange(1,11),"MEAN SQUARED ERROR":error_list})

K FOLD CV


,DEGREE,MEAN SQUARED ERROR
0,1,27.439934
1,2,21.235840
2,3,21.336606
3,4,21.353887
4,5,20.905641
5,6,20.780115
6,7,20.968094
7,8,21.077654
8,9,21.037602
9,10,20.973329


## 5.3.4 The Bootstrap

In [27]:
portfolio = pd.read_csv(r'E:\programming\dataset\Into_to_statstical_learning\Portfolio.csv',index_col=0)
print(portfolio.shape)
portfolio.head()

(100, 2)


,X,Y
1,-0.895251,-0.234924
2,-1.562454,-0.885176
3,-0.417090,0.271888
4,1.044356,-0.734198
5,-0.315568,0.841983


In [25]:
# we first define a function equivalent to func alpha defined in the Lab, i would recommend you to go through the function in lab
# This function takes two arguements, data and indeces, this indices are used to calculate the estimate for alpha for this bootstrap

In [65]:
def alpha(data,index):
    X = data['X'].loc[index]
    y = data['Y'].loc[index]
    
    return (np.var(y) - np.cov(X,y)[0][1]) / (np.var(X) + np.var(y) - 2*(np.cov(X,y)[0][1]))

In [66]:
# equivalent to sample function in lab
def get_indices(data,num_samples):
    return  np.random.choice(data.index, num_samples, replace=True)

In [72]:
alpha(portfolio,np.arange(1,100))

0.5771059047399711

In [73]:
np.random.seed(2)
alpha(portfolio,get_indices(portfolio,100))

0.6491078066222179

In [84]:
# there is no built in function like boot, so, we will define one
def boot(data,func,R):
    estimates = []
    for i in range(R):
        estimates.append(func(data,get_indices(data,100)))
    bootstrap_statistics = {'estimated_value':np.mean(estimates),'std_error':np.std(estimates)}   
    return bootstrap_statistics

In [89]:
np.random.seed(0)
results = boot(portfolio,alpha,1000)
results

{'estimated_value': 0.580591309592219, 'std_error': 0.08980409161302537}

### Estimating the Accuracy of a Linear Regression Model


In [111]:
# auto data used earlier in the notebook
data = data.reset_index()
data.head()

,index,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [149]:
# similar to boot.fn in lab

def get_estimates(data,index):
    X = data['horsepower'].loc[index]
    y = data['mpg'].loc[index]
    
    lr = LinearRegression()
    lr.fit(X.to_frame(),y)
    intercept = lr.intercept_
    coef = lr.coef_
    return [intercept,coef]

In [150]:
get_estimates(data,np.arange(0,392))

[39.93586102117047, array([-0.15784473])]

In [151]:
#modifying the boot mentioned that we used earlier
def boot(data,func,R):
    intercept = []
    coeff = []
    for i in range(R):
        intercept.append(func(data,get_indices(data,100))[0])
        coeff.append(func(data,get_indices(data,100))[1]) 
    intercept_statistics = {'estimated_value':np.mean(intercept),'std_error':np.std(intercept)}   
    coeff_statistices = {'estimated_value':np.mean(coeff),'std_error':np.std(coeff)}   
    return {'intercept':intercept_statistics,'coeff_statistices':coeff_statistices}

In [152]:
results = boot(data,get_estimates,1000)

In [153]:
print('Result for intercept ',results['intercept'])
print('Result for coefficient term ',results['coeff_statistices'])

Result for intercept  {'estimated_value': 39.971231978520734, 'std_error': 1.7370193344068778}
Result for coefficient term  {'estimated_value': -0.15881709682801645, 'std_error': 0.014778176069058804}


In [143]:
# for bootstraping we have std error 1.69 for intercept, and 0.0144 for ceoff

In [148]:
# for lets see what the model predicts
import statsmodels.api as sm
X = data['horsepower']
y = data['mpg']

X = sm.add_constant(X)
results = sm.OLS(y,X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     599.7
Date:                Wed, 08 Jul 2020   Prob (F-statistic):           7.03e-81
Time:                        11:03:19   Log-Likelihood:                -1178.7
No. Observations:                 392   AIC:                             2361.
Df Residuals:                     390   BIC:                             2369.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.9359      0.717     55.660      0.0

In [155]:
# standard error are less for estimations using model
# But still bootstrap estimates are mode preices, because they don't rely on assumptions, while there is a lot of assumptions
# when calculating std errors using sm(model)

In [158]:
# Adding a quad term
# similar to boot.fn in lab
data['horsepower_2'] = data['horsepower']**2

def get_estimates(data,index):
    X = data[['horsepower','horsepower_2']].loc[index]
    y = data['mpg'].loc[index]
    
    lr = LinearRegression()
    lr.fit(X,y)
    intercept = lr.intercept_
    coef = lr.coef_
    return [intercept,coef]

get_estimates(data,np.arange(0,392))

#modifying the boot mentioned that we used earlier
def boot(data,func,R):
    intercept = []
    coeff_1 = []
    coeff_2 = []
    for i in range(R):
        intercept.append(func(data,get_indices(data,100))[0])
        coeff_1.append(func(data,get_indices(data,100))[1][0]) 
        coeff_2.append(func(data,get_indices(data,100))[1][1])
    intercept_statistics = {'estimated_value':np.mean(intercept),'std_error':np.std(intercept)}   
    coeff_1_statistices = {'estimated_value':np.mean(coeff_1),'std_error':np.std(coeff_1)}   
    coeff_2_statistices = {'estimated_value':np.mean(coeff_2),'std_error':np.std(coeff_2)}   
    return {'intercept':intercept_statistics,'coeff_1_statistices':coeff_1_statistices,'coeff_2_statistics':coeff_2_statistices}

results = boot(data,get_estimates,1000)

print('Result for intercept ',results['intercept'])
print('Result for coefficient term horsepower',results['coeff_1_statistices'])
print('Result for coefficient term horsepower**2',results['coeff_2_statistics'])


# for bootstraping we have std error 1.69 for intercept, and 0.0144 for ceoff

# for lets see what the model predicts
import statsmodels.api as sm
X = data[['horsepower','horsepower_2']]
y = data['mpg']

X = sm.add_constant(X)
results = sm.OLS(y,X).fit()
print(results.summary())

Result for intercept  {'estimated_value': 57.22194337269758, 'std_error': 4.268750160472158}
Result for coefficient term horsepower {'estimated_value': -0.47175109701024376, 'std_error': 0.07214757143694592}
Result for coefficient term horsepower**2 {'estimated_value': 0.0012511071230436879, 'std_error': 0.00026005778901839197}
                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     428.0
Date:                Wed, 08 Jul 2020   Prob (F-statistic):           5.40e-99
Time:                        11:20:43   Log-Likelihood:                -1133.2
No. Observations:                 392   AIC:                             2272.
Df Residuals:                     389   BIC:                             2284.
Df Model:                           2 

C:\Users\Lenovo\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
